In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#setting
prompt_style = 'AutoCOT1'
data_count = 1
examle_count = 10


In [ ]:
# Configs
import config 

In [ ]:
# Run generator.js
import subprocess

params = ["200", "200"] # Enter the number of data. each prameter indicate the number of small, large map.

subprocess.run(["node", "generator.js"] + params)

In [ ]:
# Run placer.py
import placer

maps = placer.load_maps()
for map in maps:
    placer.assign_parameters(map, enemy_density=0.05, cohesion=0.3, treasure_density=0.01, range_multiplier=2, boss=True)
    placer.modify_map(map, group_min_dist=10, flag_try_count=50, enemy_sparsity=3)
placer.save_maps(maps)

In [ ]:
# Run labeler.py
import labeler

labeler.label(placed_file_path=config.PLACED_PATH, labelled_file_path=config.LABELED_PATH, file_count=4, difficulty_curve_interval=5)

In [ ]:
# Generate preprocessed
from preprocessed_data_generator import preprocessed_data_generate

preprocessed_data_generate(data_count, examle_count, prompt_style)




In [ ]:
# Run comparator.py
import comparator

comparator.compare(preprocessed_path=config.PREPROCESSED_PATH, compared_path=config.COMPARED_PATH, file_count=1)

In [ ]:
# Get compared file and calculate difference

compared_list = placer.load_json_files(config.COMPARED_PATH)
diff_dict = dict()

params = [
    "map_size",
    "room_count",
    "enemy_count",
    "treasure_count"
]

for param in params:
    diff_dict[param] = list()

for compared in compared_list:
    for param in params:
        try:
            diff_dict[param].append([abs(float(compared["after_params"][param][i]) - float(compared["before_params"][param][i])) for i in range(len(compared["after_params"][param]))])
        except:
            diff_dict[param].append(abs(float(compared["after_params"][param]) - float(compared["before_params"][param])))

In [ ]:
# Calculate mean and standard deviation
import numpy as np

mean_dict = dict()
std_dict = dict()

for param in params:
    try:
        mean_dict[param] = np.mean(np.array(diff_dict[param]), axis=0)
        std_dict[param] = np.std(np.array(diff_dict[param]), axis=0)
    except:
        mean_dict[param] = np.mean(diff_dict[param])
        std_dict[param] = np.std(diff_dict[param])

In [ ]:
# Print mean and standard deviation
print(mean_dict)
print(std_dict)

In [ ]:
# Calculate playability

playability_list = list()
for i in range(len(compared_list)):
    playability_list.append(compared_list[i]["after_params"]["playability"])

playability_mean = np.mean(playability_list)

In [ ]:
# Print playability
print(playability_mean)